In [ ]:
from os import listdir
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import cv2
from glob import glob
import os
from sklearn.preprocessing import MinMaxScaler
from sys import path
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, AveragePooling2D, GlobalMaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D , MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adam, Adadelta
from keras.layers import Input
from keras.models import Model
from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from sklearn.metrics import mean_squared_error, mean_absolute_error
import keras.backend as K

from sklearn.preprocessing import MinMaxScaler
from keras.layers import Conv2DTranspose , Concatenate, UpSampling2D
from google.colab import drive
drive.mount('/content/drive/' , force_remount=True)

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3, Xception


def augumentaion(folder , dim):
  X = []
  img_file = cv2.imread( folder +'test_img.jpg')
  if img_file is not None:
      img_orginal=cv2.resize(img_file,dim)
      img_arr = np.asarray(img_orginal)
      X.append(img_arr)
      
  else:
    print('Not Image')
  X_new=np.asarray(X)
  return X_new
Input_path= "/content/drive/MyDrive/Codes/"
dim=(224,224)
X = augumentaion(Input_path, dim)
def VGG():
    inputShape=tuple(list(dim)+[3])
    inputs = Input(shape=inputShape)
    base_model = VGG16(weights=None, include_top=False, input_shape=inputShape)
    outputs = base_model.output
    outputs = Conv2D(400, kernel_size= (1,1))(outputs)
    outputs = Flatten(name='flatten')(outputs)
    outputs = Dense(160)(outputs)
    outputs= Activation("relu")(outputs)
    outputs = BatchNormalization()(outputs)
    prediction = Dense(3,activation="linear")(outputs)
    model_vgg16 = Model(base_model.input, outputs=prediction , name="Vgg16")
    return model_vgg16
opt = Adam(learning_rate=0.0001)
metric= RootMeanSquaredError()
lossfunc='mse'
VGG = VGG() 
VGG.compile(optimizer=opt, loss=lossfunc, metrics=metric)
def Resnet():
    inputShape=tuple(list(dim)+[3])
    inputs = Input(shape=inputShape)
    base_model = ResNet50(weights=None, include_top=False, input_shape=inputShape)
    outputs = base_model.output
    outputs = Conv2D(240, kernel_size= (1,1))(outputs)
    outputs = Flatten(name='flatten')(outputs)
    outputs = Dense(992)(outputs)
    outputs= Activation("relu")(outputs)
    outputs = Dense(992)(outputs)
    outputs= Activation("relu")(outputs)
    outputs = Dense(32)(outputs)
    outputs= Activation("relu")(outputs)
    outputs = BatchNormalization()(outputs)
    prediction = Dense(3,activation="linear")(outputs)
    model_resnet50 = Model(base_model.input, outputs=prediction , name="Resnet50")

    return model_resnet50
lr = 0.01
opt = Adam(learning_rate=lr)
# compile the model
metric= RootMeanSquaredError()
lossfunc='mse'
Resnet = Resnet() 
Resnet.compile(optimizer=opt, loss=lossfunc, metrics=metric)
os.chdir('/content/drive/MyDrive/Codes')
import h5py
from tensorflow.keras.models import load_model
model1 = Resnet
model1.load_weights('Resnet_weights.h5')
model2 = VGG
model2.load_weights('Vgg16_weights.h5')
def extractFeatures(X,model):
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    #print(model.summary())

    # feature extraction from photos
    features = []
    i=0


    for x in X:
        x = x.reshape((1, x.shape[0], x.shape[1], x.shape[2]))
        x = preprocess_input(x)
        features.append(model.predict(x, verbose = 0))
        i+=1
    features=np.array(features)
    
    return features
feats1 = extractFeatures(X,model1)
feats2 = extractFeatures(X,model2)
feats1=np.array(feats1)
XNew1=feats1.reshape(feats1.shape[0],feats1.shape[2])
feats2=np.array(feats2)
XNew2=feats2.reshape(feats2.shape[0],feats2.shape[2])
XNew=np.concatenate((XNew1, XNew2), axis=1)
import joblib
os.chdir('/content/drive/MyDrive/Codes')
reg_model1 = joblib.load("./Adaboost.joblib")
reg_model2 = joblib.load("./Randomforest.joblib")
Out1 = reg_model1.predict(XNew)
Out2 = reg_model2.predict(XNew)
print('Normalized Adaboost Output')
print(Out1)
print('Normalized Randomforest Output')
print(Out2)


ValueError: ignored